In [30]:
from google.colab import files
uploaded = files.upload()

In [31]:
import json
import web_scraper as webScraper
import category_utils as util

In [32]:
with open("category_tree.json", "r") as f:
    category_tree = json.load(f)

In [41]:
print(" > ".join(util.find_category_path(category_tree, "addiction_rehabilitation_center")))

health_and_medical > rehabilitation_center > addiction_rehabilitation_center


In [28]:
find_top_level_category

predicted = "eat_and_drink"
true_label = "cafe"

print(util.is_prediction_correct(predicted, true_label))
# Should return True if "ethiopian_restaurant" is in the subtree of "eat_and_drink"


True


In [17]:
import sbert_classifier as clf

model = clf.SentenceTransformer('all-MiniLM-L6-v2')

In [27]:
from category_keywords import category_keywords

embeddings = clf.embed_tree_nodes_by_layer(category_keywords, model)

In [21]:
desc = "Montessori Children's CenterMontessori Children's CenterHomeMontessori Children's Center's goal is to help children develop and nurture their intrinsic love of learning, their independence, and their sense of community.Montessori Children's CenterWelcome to Montessori Children’s CenterHere at MCC, we believe in letting the child’s spirit guide us. It is our goal to help children develop and nurture their intrinsic love of learning, their independence, and their sense of community. In the process, we hope to aid them in becoming lifelong learners.TAKE A VIRTUAL TOUR OF THE SCHOOLLocated in picturesque Parkmerced in the southwest corner of San Francisco, the school is housed in state of the art facilities in a quiet neighborhood. Founded in 1976, the MCC has been serving the children and families of San Francisco for forty years.Download our BrochureClick for more informationMontessori Children’s Center80 Juan Bautista Circle | San Francisco, CA 94132 |415.333.4410Located in the southwest corner of San Francisco, near San Francisco State University in the Parkmerced Complex.View Larger MapHome|About Us|ProgramsAdmissions|Calendar|Contact Us| CareersCopyright 2023Montessori Children’s CenterSan Francisco, CA 94132"

pred_path, top_n_per_layer, is_ambiguous, ambiguous_layers = clf.classify_with_layered_tree_top_n(
    description = desc,
    tree = category_keywords,
    embeddings = embeddings,
    model = model,
    rule_weight = 0.6,
    top_n = 3,
    ambiguity_threshold = 0.1
)

print(f'Final predicted path: {pred_path}\n')

if is_ambiguous:
    print("Ambiguity detected at level(s):", [lvl + 1 for lvl in ambiguous_layers])
    print("Consider fallback, user confirmation, or LLM-based disambiguation.\n")

print("\nTop candidates at each level:")
for level, candidates in enumerate(top_n_per_layer):
    print(f"Level {level + 1}:")
    for name, score in candidates:
        print(f"  {name} - score: {score:.4f}")
    print()


Final predicted path: education > college_university


Top candidates at each level:
Level 1:
  education - score: 1.0000
  attractions_and_activities - score: 0.0887
  accommodation - score: 0.0749

Level 2:
  college_university - score: 1.0000
  tutoring_center - score: 0.4568
  educational_camp - score: 0.4455

